In [96]:
import torch

variables and parameters

In [97]:
momentum = 0.1
n_features = 3
length = 10
batch_size = 32
eps = 1e-5
affine=True
track_running_stats=True
device = torch.device("cpu")
dtype = torch.float64

shape1 = (batch_size, n_features)
shape2 = (batch_size, n_features, length)


torch.manual_seed(0)
x1 = torch.randn(shape1, dtype=dtype, requires_grad=True, device=device)
x2 = torch.randn(shape2, dtype=dtype, requires_grad=True, device=device)

rank 2 tensors:

In [117]:
norm = torch.nn.BatchNorm1d(n_features, momentum=momentum, eps=eps, affine=affine, track_running_stats=track_running_stats, dtype=dtype)
print(f"before normalization: running mean: {norm.running_mean}")
print(f"before normalization: running var: {norm.running_var}")
out1 = norm(x1)
running_mean_torch, running_var_torch = norm.running_mean, norm.running_var
print(f"after normalization: running mean: {norm.running_mean}")
print(f"after normalization: running var: {norm.running_var}")

before normalization: running mean: tensor([0., 0., 0.], dtype=torch.float64)
before normalization: running var: tensor([1., 1., 1.], dtype=torch.float64)
after normalization: running mean: tensor([ 0.0005, -0.0229,  0.0165], dtype=torch.float64)
after normalization: running var: tensor([1.0395, 0.9768, 1.0161], dtype=torch.float64)


In [118]:
mean = x1.mean(dim=0, keepdim=True)
var = x1.var(dim=0, unbiased=False, keepdim=True)
out1_man = (x1 - mean) / torch.sqrt(var + eps)
print(f"normalization correct: {torch.allclose(out1_man, out1)}")

n = torch.tensor(x1.shape[0])
n = batch_size

running_mean_man = torch.zeros(mean.shape, dtype=dtype)
running_var_man = torch.ones(var.shape, dtype=dtype)

running_mean_man = (1 - momentum) * running_mean_man + momentum * mean
running_var_man = (1 - momentum) * running_var_man + momentum * var * n / (n - 1)

print(f"running mean correct: {torch.allclose(running_mean_man, running_mean_torch)}") # type: ignore
print(f"running var correct: {torch.allclose(running_var_man, running_var_torch)}") # type: ignore
running_var_man.detach().numpy(), running_var_torch.detach().numpy()

normalization correct: True
running mean correct: True
running var correct: True


(array([[1.03948093, 0.97679327, 1.01605829]]),
 array([1.03948093, 0.97679327, 1.01605829]))

In [119]:
norm2 = torch.nn.BatchNorm1d(n_features, momentum=momentum, eps=eps, affine=affine, track_running_stats=track_running_stats, dtype=dtype)
print(f"before normalization: running mean: {norm2.running_mean}")
print(f"before normalization: running var: {norm2.running_var}")
out2 = norm2(x2)
running_mean_torch2, running_var_torch2 = norm2.running_mean, norm2.running_var
print(f"after normalization: running mean: {norm2.running_mean}")
print(f"after normalization: running var: {norm2.running_var}")

before normalization: running mean: tensor([0., 0., 0.], dtype=torch.float64)
before normalization: running var: tensor([1., 1., 1.], dtype=torch.float64)
after normalization: running mean: tensor([-0.0055,  0.0048,  0.0009], dtype=torch.float64)
after normalization: running var: tensor([1.0006, 0.9999, 0.9947], dtype=torch.float64)


In [ ]:
mean = x2.mean(dim=(0,2))
var = x2.var(dim=(0,2), unbiased=False)
out2_man = (x1 - mean) / torch.sqrt(var + eps)
print(f"normalization correct: {torch.allclose(out1_man, out1)}")

n = torch.tensor(x1.shape[0] * x1.shape[2])
# n = batch_size

running_mean_man = torch.zeros(mean.shape, dtype=dtype)
running_var_man = torch.ones(var.shape, dtype=dtype)

running_mean_man = (1 - momentum) * running_mean_man + momentum * mean
running_var_man = (1 - momentum) * running_var_man + momentum * var * n / (n - 1)

print(f"running mean correct: {torch.allclose(running_mean_man, running_mean_torch)}") # type: ignore
print(f"running mean correct: {torch.allclose(running_var_man, running_var_torch)}") # type: ignore
running_var_man.detach().numpy(), running_var_torch.detach().numpy()

In [149]:
mean2 = x2.mean(dim=(0,2), keepdim=True)
var2 = x2.var(dim=(0,2), unbiased=False, keepdim=True)
print(x2.shape, mean2.shape, var2.shape)
out2_man = (x2 - mean2) / torch.sqrt(var2 + eps)
print(f"normalization correct: {torch.allclose(out2_man, out2)}")

n2 = torch.tensor(x2.shape[0] * x2.shape[2])
# n = batch_size

running_mean_man2 = torch.zeros(mean2.shape, dtype=dtype)
running_var_man2 = torch.ones(var2.shape, dtype=dtype)

running_mean_man2 = (1 - momentum) * running_mean_man2 + momentum * mean2
running_var_man2 = (1 - momentum) * running_var_man2 + momentum * var2 * n2 / (n2 - 1)

print(f"running mean correct: {torch.allclose(running_mean_man2.squeeze(), running_mean_torch2)}") # type: ignore
print(f"running mean correct: {torch.allclose(running_var_man2.squeeze(), running_var_torch2)}") # type: ignore
# running_var_man2.squeeze().detach().numpy(), running_var_torch2.detach().numpy()
running_mean_man2.squeeze().detach().numpy(), running_mean_torch2.detach().numpy()

torch.Size([32, 3, 10]) torch.Size([1, 3, 1]) torch.Size([1, 3, 1])
normalization correct: True
running mean correct: True
running mean correct: True


(array([-0.00548487,  0.00478185,  0.00094317]),
 array([-0.00548487,  0.00478185,  0.00094317]))